In [ ]:
#setting os path to import scripts
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# imports
from deep_lens_modeling.network_predictions import generate_sequential_predictions
from scipy.stats import multivariate_normal
from Scripts import network_predictions, fermat_potentials
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines
import statistics

# lenstronomy stuff
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.Plots import lens_plot

In [ ]:
#autoreload imports
%load_ext autoreload
%autoreload 2

In [ ]:
sample_num= 101
path = module_path+'/'
config_path = 'Configs'
image_path = 'Images_for_Network'
y_test, y_pred_wop, std_pred_wop, prec_pred_wop, y_pred_wp, std_pred_wp, prec_pred_wp, y_pred_wpl, std_pred_wpl, prec_pred_wpl = network_predictions.Predictions(sample_num, path, config_path, image_path)

In [ ]:
#Finding image positions
x_im_test = []
y_im_test = []

for i in range(sample_num-1):
    x_im_test_val, y_im_test_val = fermat_potentials.fermat.image_positions_from_y_pred(y_test[i])
    x_im_test.append(x_im_test_val)
    y_im_test.append(y_im_test_val)

In [ ]:
x_im_pred_wop = []
y_im_pred_wop = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.fermat.image_positions_from_y_pred(y_pred_wop[i])
    x_im_pred_wop.append(x_im_pred_val)
    y_im_pred_wop.append(y_im_pred_val)

In [ ]:
x_im_pred_wp = []
y_im_pred_wp = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.fermat.image_positions_from_y_pred(y_pred_wp[i])
    x_im_pred_wp.append(x_im_pred_val)
    y_im_pred_wp.append(y_im_pred_val)

In [ ]:
x_im_pred_wpl = []
y_im_pred_wpl = []

for i in range(sample_num-1):
    x_im_pred_val, y_im_pred_val = fermat_potentials.fermat.image_positions_from_y_pred(y_pred_wpl[i])
    x_im_pred_wpl.append(x_im_pred_val)
    y_im_pred_wpl.append(y_im_pred_val)

In [ ]:
params_dist_wop = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wop[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wop[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wop.append(params_dist_val)

In [ ]:
params_dist_wp = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wp[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wp[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wp.append(params_dist_val)

In [ ]:
params_dist_wpl = []
for i in range(sample_num-1):
    covariance_matrix = np.diag(std_pred_wpl[i]**2)
    params_dist_val = multivariate_normal.rvs(mean=y_pred_wpl[i],cov=covariance_matrix,size=1000)
    #plt.hist(params_dist[:,3])
    params_dist_wpl.append(params_dist_val)

In [ ]:
# Plotting histograms
n = 0
color = ['slategray', 'firebrick', 'goldenrod']
label = ['Without Perturber', 'With Perturber', 'With Perturber Light']

for i in range(sample_num-1):
    truth_fermat_potentials = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test[i], x_im_test[i], y_im_test[i]))
    
    #Find location of largest time delay and plot one set of hitograms for each quad
    if truth_fermat_potentials.shape == (4,):
        shape = 3
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC','AD']
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
        shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        counts,bins,_ = axs.hist(sample_wop_arr[abs_max_loc],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_wp_arr[abs_max_loc],bins=20, histtype=u'step', color='firebrick')
        counts,bins,_ = axs.hist(sample_wpl_arr[abs_max_loc],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr[abs_max_loc],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False, fontsize=5,loc='upper left')

    elif truth_fermat_potentials.shape == (3,):
        shape = 2
        fig,axs = plt.subplots(1,figsize=(12,4))
        im_labels = ['AB','AC']
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
            shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        counts,bins,_ = axs.hist(sample_wop_arr[abs_max_loc],bins=20, histtype=u'step', color='slategray')
        counts,bins,_ = axs.hist(sample_wp_arr[abs_max_loc],bins=20, histtype=u'step', color='firebrick')
        counts,bins,_ = axs.hist(sample_wpl_arr[abs_max_loc],bins=20, histtype=u'step', color='goldenrod')
        axs.vlines(truth_arr[abs_max_loc],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{'+im_labels[abs_max_loc]+'}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False, fontsize=5,loc='upper left')

    elif truth_fermat_potentials.shape == (2,):
        fig,axs = plt.subplots(1,figsize=(12,4))
        sampled_fermat_potentials_wop,sampled_fermat_potentials_wp,sampled_fermat_potentials_wpl = fermat_potentials.fermat.fermat_potential_arrays(
            params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i])
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,1],bins=20, histtype=u'step',
                                        color='slategray')
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,1],bins=20, histtype=u'step',
                                        color='firebrick')
        counts,bins,_ = axs.hist(sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,1],bins=20, histtype=u'step',
                                        color='goldenrod')
        axs.vlines(truth_fermat_potentials[0]-truth_fermat_potentials[1],0,np.max(counts),zorder=200,color='red')
        axs.set_title('$\Delta\phi_{AB}$')
        axs.legend(handles=[mlines.Line2D([], [], color=color[i], label=label[i]) for i in range(0,3)],frameon=False,
                fontsize=5,loc='upper left')

    else:
        print('Index:'+str(i))
        print('Sampled fermat potentials WoP size: '+str(sampled_fermat_potentials_wop[i].shape))
        print('Sampled fermat potentials WP size: '+str(sampled_fermat_potentials_wp[i].shape))
        print('Sampled fermat potentials WPL size: '+str(sampled_fermat_potentials_wpl[i].shape))
        print('Truth fermat potentials size: '+str(truth_fermat_potentials.shape))
        n = n+1

print('Missing fermat potential differences: '+str(n))

In [ ]:
test = sample_wop_arr[abs_max_loc]
#print(test)
median = statistics.median(test)
fig,axs = plt.subplots(1,figsize=(12,4))
counts,bins,_ = axs.hist(sample_wop_arr[abs_max_loc],bins=20, histtype=u'step', color='slategray')
axs.vlines(median,0,np.max(counts),zorder=200,color='red')
axs.vlines(truth_arr[abs_max_loc],0,np.max(counts),zorder=200,color='black')
print(median-truth_arr[abs_max_loc])

### Calculate Metrics

In [ ]:
#Accurace
acc_arr_wop = []
acc_arr_wp = []
acc_arr_wpl = []

for i in range(sample_num-1):
    truth_fermat_potentials = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test[i], x_im_test[i], y_im_test[i]))

    if truth_fermat_potentials.shape == (4,):
        shape = 3
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
            shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        median_wop = statistics.median(sample_wop_arr[abs_max_loc])
        median_wp = statistics.median(sample_wp_arr[abs_max_loc])
        median_wpl = statistics.median(sample_wpl_arr[abs_max_loc])
        truth = truth_arr[abs_max_loc]
        acc_arr_wop.append(median_wop-truth)
        acc_arr_wp.append(median_wp-truth)
        acc_arr_wpl.append(median_wpl-truth)

    elif truth_fermat_potentials.shape == (3,):
        shape = 2
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
            shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        median_wop = statistics.median(sample_wop_arr[abs_max_loc])
        median_wp = statistics.median(sample_wp_arr[abs_max_loc])
        median_wpl = statistics.median(sample_wpl_arr[abs_max_loc])
        truth = truth_arr[abs_max_loc]
        acc_arr_wop.append(median_wop-truth)
        acc_arr_wp.append(median_wp-truth)
        acc_arr_wpl.append(median_wpl-truth)

    elif truth_fermat_potentials.shape == (2,):
        sampled_fermat_potentials_wop,sampled_fermat_potentials_wp,sampled_fermat_potentials_wpl = fermat_potentials.fermat.fermat_potential_arrays(
            params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i])
        sample_wop = sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,1]
        sample_wp = sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,1]
        sample_wpl = sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,1]
        median_wop = statistics.median(sample_wop)
        median_wp = statistics.median(sample_wp)
        median_wpl = statistics.median(sample_wpl)
        truth = truth_fermat_potentials[0]-truth_fermat_potentials[1]
        acc_arr_wop.append(median_wop-truth)
        acc_arr_wp.append(median_wp-truth)
        acc_arr_wpl.append(median_wpl-truth)

acc_metric_wop = statistics.mean(acc_arr_wop)
acc_metric_wp = statistics.mean(acc_arr_wp)
acc_metric_wpl = statistics.mean(acc_arr_wpl)
print('Accuracy of WoP: '+str(acc_metric_wop))
print('Accuracy of WP: '+str(acc_metric_wp))
print('Accuracyof WPL: '+str(acc_metric_wpl))

In [ ]:
#Precision

prec_arr_wop = []
prec_arr_wp = []
prec_arr_wpl = []

for i in range(sample_num-1):
    truth_fermat_potentials = (fermat_potentials.fermat.fermat_potential_at_image_positions(y_test[i], x_im_test[i], y_im_test[i]))

    if truth_fermat_potentials.shape == (4,):
        shape = 3
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
            shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        variance_wop = np.var(sample_wop_arr[abs_max_loc])
        variance_wp = np.var(sample_wp_arr[abs_max_loc])
        variance_wpl = np.var(sample_wpl_arr[abs_max_loc])

        prec_arr_wop.append(1/variance_wop)
        prec_arr_wp.append(1/variance_wp)
        prec_arr_wpl.append(1/variance_wpl)

    elif truth_fermat_potentials.shape == (3,):
        shape = 2
        sample_wop_arr,sample_wp_arr,sample_wpl_arr,truth_arr,abs_max_loc = fermat_potentials.fermat.largest_truth_value(
            shape,params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i],truth_fermat_potentials)
        variance_wop = np.var(sample_wop_arr[abs_max_loc])
        variance_wp = np.var(sample_wp_arr[abs_max_loc])
        variance_wpl = np.var(sample_wpl_arr[abs_max_loc])

        prec_arr_wop.append(1/variance_wop)
        prec_arr_wp.append(1/variance_wp)
        prec_arr_wpl.append(1/variance_wpl)

    elif truth_fermat_potentials.shape == (2,):
        sampled_fermat_potentials_wop,sampled_fermat_potentials_wp,sampled_fermat_potentials_wpl = fermat_potentials.fermat.fermat_potential_arrays(
            params_dist_wop[i],params_dist_wp[i],params_dist_wpl[i],y_test[i],x_im_test[i],y_im_test[i])
        sample_wop = sampled_fermat_potentials_wop[:,0]-sampled_fermat_potentials_wop[:,1]
        sample_wp = sampled_fermat_potentials_wp[:,0]-sampled_fermat_potentials_wp[:,1]
        sample_wpl = sampled_fermat_potentials_wpl[:,0]-sampled_fermat_potentials_wpl[:,1]
        variance_wop = np.var(sample_wop)
        variance_wp = np.var(sample_wp)
        variance_wpl = np.var(sample_wpl)

        prec_arr_wop.append(1/variance_wop)
        prec_arr_wp.append(1/variance_wp)
        prec_arr_wpl.append(1/variance_wpl)

prec_metric_wop = statistics.mean(prec_arr_wop)
prec_metric_wp = statistics.mean(prec_arr_wp)
prec_metric_wpl = statistics.mean(prec_arr_wpl)
print('Precision of WoP: '+str(prec_metric_wop))
print('Precision of WP: '+str(prec_metric_wp))
print('Precision of WPL: '+str(prec_metric_wpl))